<a href="https://colab.research.google.com/github/quent1fvr/Infection_pulmonaire/blob/main/model_CNN3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0 - Importation des données et librairies**



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import tensorflow
import keras
from tensorflow.keras import models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import TimeDistributed
from keras.layers import Conv3D, BatchNormalization,MaxPooling3D, GlobalMaxPool3D
from keras.layers import TimeDistributed, GRU, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix




In [3]:
hf_normal= h5py.File("/content/drive/MyDrive/UV PROJET P6/Dataset_normal.hdf5", "r")
hf_malade= h5py.File("/content/drive/MyDrive/UV PROJET P6/Dataset_malade.hdf5", "r")

Data_normal = np.array(hf_normal["dataset_1"][:]) #dataset_name is same as hdf5 object name 
Data_malade = np.array(hf_malade["dataset_2"][:])

# **1 - Etude des données et création des labels**



In [4]:
Data_normal.shape , Data_malade.shape

((55720, 48, 48), (54600, 48, 48))

In [5]:
Data_normal = Data_normal.reshape(Data_normal.shape[0]//70,70,48,48) # work on 1576 samples of 70 scans of size 48*48
Data_malade = Data_malade.reshape(Data_malade.shape[0]//70,70,48,48) # work on 1576 samples of 70 scans of size 48*48
Data = np.concatenate((Data_normal,Data_malade))
Data.shape

(1576, 70, 48, 48)

Le jeu de données est composé de séquences de 70 images par scan, c'est pourquoi on atteibue un label pour chaque séquence :
- 0 : La personne est négative au COVID. 
- 1 : La personne est positive au COVID

In [20]:
Y_normal = np.zeros(Data_normal.shape[0]//70,dtype = np.int8) # 70 images per scan
Y_malade = np.ones(Data_malade.shape[0]//70,dtype = np.int8)
Y = np.concatenate((Y_normal,Y_malade))
Y.shape


(1576,)

# **2 - Création d'un modèle CNN3D**




In [33]:
def build_convnet(shape=(70,48, 48)):
    momentum = .9
    model = keras.Sequential()
    model.add(Conv3D(64, (3,3,3), input_shape=shape,
        padding='same', activation='relu'))
    model.add(Conv3D(64, (3,3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPooling3D())
    
    model.add(Conv3D(128, (3,3,3), padding='same', activation='relu'))
    model.add(Conv3D(128, (3,3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(GlobalMaxPool3D())
    return model

In [34]:
def action_model(shape=(70,48, 48,1), nbout=1):
    convnet = build_convnet(shape)
    
    model = keras.Sequential()
    model.add(convnet)
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='sigmoid'))
    return model

In [18]:
# some global params
SIZE = (48, 48)
NBFRAME = 70
BS = 8
CHANNEL = 1

In [10]:
INSHAPE=(70,48,48,1)
model = action_model(INSHAPE, nbout=1)
optimizer = tensorflow.keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'binary_crossentropy',
    metrics=['acc']
)

On fixe la graine aléatoire afin que l'on puisse avoir des résultats similaires en relançant l'apprentissage : 

In [11]:
def fix_seed(seed):
    tensorflow.random.set_seed(seed)

SEED = 42
fix_seed(SEED)

On utilise les callbacks EarlyStop et ModelCheckpoint pour optimiser nos résultats : 

In [12]:
my_callbacks = [
    tensorflow.keras.callbacks.EarlyStopping(monitor = 'val_acc',
                          min_delta = 0,
                          patience = 20,
                          verbose = 1,
                          restore_best_weights = True),
    tensorflow.keras.callbacks.ModelCheckpoint(filepath='/tmp/myModel.h5')
]



# **3 - Apprentissage et interprétation des résultats**


In [13]:
model.fit(Data,Y,epochs=50, validation_split=0.2, batch_size=BS, callbacks=my_callbacks)

Epoch 1/50
158/158 [==============================] - 104s 552ms/step - loss: 0.9900 - acc: 0.5365 - val_loss: 1.1575 - val_acc: 0.0095
Epoch 2/50
158/158 [==============================] - 88s 556ms/step - loss: 0.7015 - acc: 0.5857 - val_loss: 0.7884 - val_acc: 0.0095
Epoch 3/50
158/158 [==============================] - 89s 566ms/step - loss: 0.6818 - acc: 0.6198 - val_loss: 0.8968 - val_acc: 0.0095
Epoch 4/50
158/158 [==============================] - 90s 567ms/step - loss: 0.6569 - acc: 0.6310 - val_loss: 1.1736 - val_acc: 0.0095
Epoch 5/50
158/158 [==============================] - 89s 567ms/step - loss: 0.6896 - acc: 0.6198 - val_loss: 0.7153 - val_acc: 0.0000e+00
Epoch 6/50
158/158 [==============================] - 90s 567ms/step - loss: 0.6811 - acc: 0.6024 - val_loss: 0.8436 - val_acc: 0.0000e+00
Epoch 7/50
158/158 [==============================] - 89s 564ms/step - loss: 0.6315 - acc: 0.6556 - val_loss: 0.4586 - val_acc: 0.7437
Epoch 8/50
158/158 [==========================

On récupère le meilleur modèle, c'est à dire celui de l'epoch 29 

In [24]:
model.load_weights("/tmp/myModel.h5")



In [31]:
acc = model.history.history['acc']
val_acc = model.history.history['val_acc']
loss = model.history.history['loss']
val_loss = model.history.history['val_loss']

print(acc)
print(val_acc)

print(loss)
print(val_loss)

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: ignored

# **4 - Test sur un jeu de données aléatoire**


In [25]:
Data_test = []
for i in range(25):
  aleatoire = np.random.randint(0,Data_normal.shape[0])
  Data_test.append(Data_normal[aleatoire][:])
for i in range(25):
  aleatoire = np.random.randint(0,Data_malade.shape[0])
  Data_test.append(Data_malade[aleatoire][:])
Data_test = np.array(Data_test)
Data_test.shape

(50, 70, 48, 48)

In [27]:
Y_test_normal= np.zeros(25)
Y_test_malade = np.ones(25)
Y_test = np.concatenate((Y_test_normal,Y_test_malade))

In [28]:
model.evaluate(Data_test,Y_test)

2/2 [==============================] - 6s 2s/step - loss: 0.3699 - acc: 0.9400


[0.3698709011077881, 0.9399999976158142]

In [29]:
preds = np.round(model.predict(Data_test),0)
cm = confusion_matrix(Y_test, preds)
print(cm)


[[25  0]
 [ 3 22]]


Le modèle est plus efficace que les modèles 2D, il prédit avec une accuracy de 93% sur un jeu de donnée aléatoire en test.

In [4]:
hf_normal= h5py.File("/content/drive/MyDrive/UV PROJET P6/dataset_normal_methode_2.2_3.2.hdf5", "r")
hf_malade= h5py.File("/content/drive/MyDrive/UV PROJET P6/dataset_malade_methode_2.2_3.2.hdf5", "r")

Data_normal2 = np.array(hf_normal["dataset_1"][:]) #dataset_name is same as hdf5 object name 
Data_malade2 = np.array(hf_malade["dataset_2"][:])

In [5]:
Data_normal2.shape, Data_malade2.shape

((55720, 48, 48), (54600, 48, 48))

In [7]:
Data_normal2 = Data_normal.reshape(Data_normal.shape[0]//70,70,48,48) # work on 1576 samples of 70 scans of size 48*48
Data_malade2 = Data_malade.reshape(Data_malade.shape[0]//70,70,48,48) # work on 1576 samples of 70 scans of size 48*48
Data2 = np.concatenate((Data_normal2,Data_malade2))
Data2.shape

(1576, 70, 48, 48)

In [24]:
my_callbacks2 = [
    tensorflow.keras.callbacks.ModelCheckpoint(filepath='/tmp/myModel2.1.h5',    monitor="val_acc"),
    tensorflow.keras.callbacks.EarlyStopping(monitor = 'val_acc',
                          min_delta = 0,
                          patience = 10,
                          verbose = 1,
                          restore_best_weights = True)
    ]


In [12]:
INSHAPE=(70,48,48,1)
model2 = action_model(INSHAPE, nbout=1)
optimizer = tensorflow.keras.optimizers.Adam(0.001)
model2.compile(
    optimizer,
    'binary_crossentropy',
    metrics=['acc']
)

In [25]:
model2.fit(Data2,Y,epochs=30, validation_split=0.2, batch_size=BS, callbacks=my_callbacks2)

Epoch 1/50
158/158 [==============================] - 84s 531ms/step - loss: 0.6340 - acc: 0.6778 - val_loss: 0.8686 - val_acc: 0.0000e+00
Epoch 2/50
158/158 [==============================] - 88s 555ms/step - loss: 0.6692 - acc: 0.6310 - val_loss: 0.9049 - val_acc: 0.0000e+00
Epoch 3/50
158/158 [==============================] - 90s 570ms/step - loss: 0.6638 - acc: 0.6302 - val_loss: 0.9288 - val_acc: 0.0000e+00
Epoch 4/50
158/158 [==============================] - 90s 570ms/step - loss: 0.6665 - acc: 0.6214 - val_loss: 1.1203 - val_acc: 0.0000e+00
Epoch 5/50
158/158 [==============================] - 90s 570ms/step - loss: 0.6724 - acc: 0.6270 - val_loss: 0.8726 - val_acc: 0.0000e+00
Epoch 6/50
158/158 [==============================] - 90s 571ms/step - loss: 0.6617 - acc: 0.6310 - val_loss: 0.9608 - val_acc: 0.0000e+00
Epoch 7/50
158/158 [==============================] - 90s 570ms/step - loss: 0.6525 - acc: 0.6317 - val_loss: 1.0130 - val_acc: 0.0000e+00
Epoch 8/50
158/158 [=======

KeyError: ignored

In [26]:
model2.load_weights("/tmp/myModel2.1.h5")

Data_test = []
for i in range(25):
  aleatoire = np.random.randint(0,Data_normal2.shape[0])
  Data_test.append(Data_normal2[aleatoire][:])
for i in range(25):
  aleatoire = np.random.randint(0,Data_malade2.shape[0])
  Data_test.append(Data_malade2[aleatoire][:])
Data_test = np.array(Data_test)
Data_test.shape
Y_test_normal= np.zeros(25)
Y_test_malade = np.ones(25)
Y_test = np.concatenate((Y_test_normal,Y_test_malade))
model2.evaluate(Data_test,Y_test)

2/2 [==============================] - 1s 288ms/step - loss: 0.2252 - acc: 0.9600


[0.22523929178714752, 0.9599999785423279]

In [32]:
acc = model2.history.history['acc']
val_acc = model2.history.history['val_acc']
loss = model2.history.history['loss']
val_loss = model2.history.history['val_loss']

print(acc)
print(val_acc)

print(loss)
print(val_loss)

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

KeyError: ignored

In [ ]:
preds = np.round(model.predict(Data_test),0)
cm = confusion_matrix(Y_test, preds)
print(cm)
